# A Tutorial for using Condynsate
In this tutorial, we will introduce how to create a simulation environment, how to set up the animator, how to build a simple controller, and how to run a simulation that includes keyboard interactivity. In the scope of this project, we will do this by testing a simple PD controller for a powered wheel with a target pointing angle.

## Building the simulation environment
When starting a Python project, the first thing we do is import our dependencies.

In [1]:
# Import condynsate. This package is used to simulate, render, and plot dynamic systems
import condynsate

To use condynsate, we first create an instanace of a simulator. The simulator handles simulating dynamic systems, rendering them in real-time, and creating live plots that show us data from the simulation. 

Running this command will both create a simulator and open a visualization window in your default internet browser.

In [2]:
# Create an instance of the simulator with visualization and animation
sim = condynsate.Simulator(visualization=True,  # Visualization is the process of rendering your dynamic system in 3D
                           visualization_fr=15.,# This parameter is the frame rate of the visualizer (3D renderer)
                           animation=True,      # Animation is the process of live plotting data from the simulation
                           animation_fr=15.     # This parameter is the frame rate of the animator (plots)
                          )

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


Next, we load URDF files into the simulator. These URDF files encode visual, collisional, and inertial data of articulated bodies --rigid bodies joined together by joints. The articulate body format is a common way to define robotic systems.

As we load each URDF file, it will automatically appear in your visualization window. We will load 2 URDF files, one file describing the powered wheel and the other describing an arrow that we will continuously update to point in the target direction.

In [3]:
# Load the wheel
wheel_obj = sim.load_urdf(urdf_path='./wheel_vis/wheel.urdf', # The relative path that leads to the wheel .urdf file 
                          position=[0., 0., 0.],              # The (X, Y, Z) coordinates at which the wheel object is placed           
                          fixed=True,                         # The origin of the wheel (base) is not allowed to move
                          update_vis=True                     # The wheel object will be updated in the visualizer each simulation step
                         )

In [4]:
# Load the target arrow. This arrow will point in the direction that we want to rotate the wheel to.
# When first loaded, the target arrow will not be visible. This is because it is pointing in the same direction 
# as the orange arrow attached to the wheel.
target_obj = sim.load_urdf(urdf_path='./wheel_vis/target_arrow.urdf',
                           position=[0., 0., 0.6655],
                           fixed=True,
                           update_vis=True
                          )

## Setting up the animator
Now we will set up the animator. The animator draws plots in real time. We define how many suplots we want, how many artists each subplot will have, and style choices for each subplot.

There are two types of subplots that can be used: **line** and **bar**. For a line plot, each individual line requires an artist, so if you want a plot with two lines on it, ```n_artists = 2```. For a bar chart, each indivdual bar requires an artist, so if you wanted to plot 3 bars, ```n_artists = 3```. More information on how to add subplots to the animator can be found in the documentation.

In this case, we will make 3 subplots. The first subplot contains two line artists that together plot the angle of the wheel and the target angle versus time. In second subplot contains one line artist that will plot the applied torque vs time. The final subplot will contain two bar artists that will plot the value of the P and D control gains.

In [5]:
# This defines the first subplot.
# This subplot will plot the current angle of the wheel and the target angle of the wheel.
plot1, artists1 = sim.add_subplot(n_artists=2,               # How many total things that are going to drawn to the subplot (2 line charts)
                                  subplot_type='line',       # What type of subplot this is going to be
                                  title="Angles vs Time",    # The title of our subplot
                                  x_label="Time [Seconds]",  # The label to give the x axis
                                  y_label="Angles [Rad]",    # The label to give the y axis
                                  colors=["r", "b"],         # The colors of each line
                                  line_widths=[2.5, 2.5],    # The width (thickness) of each line
                                  line_styles=["-", ":"],    # The style of each line (solid and dotted)
                                  labels=["Angle", "Target"] # The name of each line in the subplot
                                 )

In [6]:
# This subplot will track the torque applied to the wheel by the axle motor
plot2, artists2 = sim.add_subplot(n_artists=1,
                                  subplot_type='line',
                                  title="Torque vs Time",
                                  x_label="Time [Seconds]",
                                  y_label="Torque [Nm]",
                                  colors=["k"],
                                  line_widths=[2.5]
                                 )

In [7]:
# This subplot is a bar chart subplot that will track constants that we set during the simulation
plot3, artists3 = sim.add_subplot(n_artists=2,                           # How many total things that are going to drawn to the subplot (2 bars)
                                  subplot_type='bar',                    # What type of subplot this is going to be
                                  title="Gains",                         # The title of our subplot
                                  x_label="Values [-]",                  # The label to give the x axis
                                  y_label="Names [-]",                   # The label to give the y axis
                                  labels=["Proportional", "Derivative"], # The name of each bar in the subplot
                                  colors=["m", "c"],                     # The face color of each bar
                                  line_widths=[1.0, 1.0],                # The width of each bar's border
                                  x_lim=[0.0, 10.0]                      # The minimum and maximum boundaries of the x axis. (Bar charts are horizontal)
                                 )                     
                            

After adding all the subplots that we want, we can now open the animator GUI. This will open a new window that shows the plot that you just defined.

In [8]:
sim.open_animator_gui()

## Design and build a controller
Here we make our controller. The controller will take some values as arguments then use them to calculate how much torque we should apply to the wheel.

kwargs is a special python function argument that allows us to pass as many arguments as we want to the controller. Each of these arguments are called "key word arguments". This means that they contain both a key and a value, exactly like the standard library dictionary. So, if we wanted to call the controller function below and pass it some value of 50 it would look like this:
```python
    controller(age = 50)
```
Above the key is 'age' and the value is 50. Therefore, if we wanted to access this data in our controller function definition, we would write:
```python
    def controller(**kwargs):
        current_age = kwargs['age']
        torque = current_age + 10
        return torque
```
By doing this, we could access the value of 50 through the kwargs key 'age'

In [9]:
# First we say what we want our target angle to be
target_angle = -3.141592654

# Now we define variables that could be useful while making a controller
var_1_P = 0.0
var_2_D = 0.0

# TODO: Make your own controller here!
def controller(**kwargs):
    # Gather our key word arguments
    error = kwargs['angle'] - kwargs['target_angle']
    d_error = kwargs['angle_vel']
    P = kwargs['var_1_P']
    D = kwargs['var_2_D']

    # Calculate the torque
    torque = -P*error - D*d_error
    
    # Return the torque we calculated
    return torque

## Run the simulation
Now we can run our simulation. This is done by calling ```sim.step()``` in a while loop. The simulator will automatically handle stepping the dynamics, updating the visualization, and plotting whatever data we send to our two subplots.

To start the simulation, press **enter** on your keyboard. To reset the simulation, press **backspace** on your keyboard. To pause the simulation, press **space** on your keyboard. 

When you're done, make sure to press **esc** to stop the simulation.

In [10]:
# Before we run the simulation, we wait for the user to press enter on their keyboard.
# By calling this function, we can keep the animator GUI responsive while waiting for IO.
sim.await_keypress(key="enter")

# Run the simulation
while(not sim.is_done):
    ###########################################################################
    # SENSOR
    # Use a sensor to collect the angle of the wheel 
    # and the angular velocity of the wheel
    state = sim.get_joint_state(urdf_obj=wheel_obj,         # The urdf id of the wheel that was generated above
                                joint_name="ground_to_axle" # The name of the joint whose state we want to measure
                               )
    angle = state['position']
    angle_vel = state['velocity']
    
    ###########################################################################
    # CONTROLLER
    # This is the section where you apply your controller.
    # Make sure to pass whatever keyword arguments you need to get it to work!
    torque = controller(angle=angle,
                        target_angle=target_angle,
                        angle_vel=angle_vel,
                        var_1_P=var_1_P,
                        var_2_D=var_2_D
                       )
    
    ###########################################################################
    # ACTUATOR
    # Apply the controller calculated torque to the wheel using an actuator.
    sim.set_joint_torque(urdf_obj=wheel_obj,          # The urdf id of the wheel that was generated above
                         joint_name="ground_to_axle", # The name of the joint to which the torque is applied
                         torque=torque,               # The amount of torque we apply
                         show_arrow=True,             # This indicates that we visualize the torque with an arrow
                         arrow_scale=0.3,             # The size of the arrow
                         arrow_offset=0.52            # The amount the arrow is offset from the center of the axle motor  
                        )
    
    ###########################################################################
    # UPDATE THE PLOTS
    # This is how we add data points to the animator
    # Each time step, we note the current time, the angle of the wheel,
    # the target angle of the wheel, and the torque applied.
    # We then send this data to the subplots and lines that we 
    # want them to be plotted on.
    sim.add_subplot_point(subplot_index=plot1,      # The subplot id on which we plot the angle vs time
                          artist_index=artists1[0], # The line id to which we send the angle vs time data point
                          x=sim.time,               # The x coordinate of the data point we send (current time)
                          y=angle                   # The y coordinate of the data point we send (current angle of the wheel)
                         )
    
    # We then repeat the above procedure for plotting the target angle
    # on the same subplot as the current angle but the second line.
    sim.add_subplot_point(subplot_index=plot1,
                          artist_index=artists1[1],
                          x=sim.time,
                          y=target_angle
                         )

    # Now, on the second plot, we plot the torque versus the time.
    sim.add_subplot_point(subplot_index=plot2,
                          artist_index=artists2[0],
                          x=sim.time,
                          y=torque
                         )

    # Finally, on the thrid plot, we plot the values of the variables P and D.
    # These are called our control gains.
    # For bar charts, the size of the bar is passed to the x argument. This is
    # because bar charts are plotted horizontally, so you change their length in 
    # the x direction.
    sim.add_subplot_point(subplot_index=plot3,
                          artist_index=artists3[0],
                          x=var_1_P 
                         )
    sim.add_subplot_point(subplot_index=plot3,
                          artist_index=artists3[1],
                          x=var_2_D
                         )
    
    ###########################################################################
    # UPDATE THE TARGET ANGLE
    # Now we allow the user to change the target angle in real time by using the keyboard
    target_angle = sim.iterate_val(curr_val=target_angle, # This says which value we are changing
                                   down_key='a',          # Which key needs to be pressed to decrement the value
                                   up_key='d',            # Which key needs to be pressed to increment the value
                                   iter_val=0.03,         # The amount by which we increment or decrement the value
                                   min_val=-3.1415927,    # The smallest value we are allowed to decrement down to
                                   max_val=3.1415927      # The largest value we are allowed to increment up to
                                  )

    # Adjust the target arrow so that it is always pointing in the target angle direction
    sim.set_joint_position(urdf_obj=target_obj,         # The urdf id of the wheel that was generated above
                           joint_name='world_to_arrow', # The name of the joint whose position we want to change
                           position=target_angle,       # The direction we want the target arrow to point in
                           physics=False                # Specifies whether or not to use physics to adjust the position.
                          )
    
     ###########################################################################
    # UPDATE THE CONTROL GAINS
    # Use the iterate_val function provided by sim to allow the user to use
    # the keyboard to modify the control gains
    var_1_P = sim.iterate_val(curr_val=var_1_P,
                              down_key='f',
                              up_key='r',
                              iter_val=0.02,
                              min_val=0,
                              max_val=10
                             )
    var_2_D = sim.iterate_val(curr_val=var_2_D,
                              down_key='g',
                              up_key='t',
                              iter_val=0.02,
                              min_val=0,
                              max_val=10
                             )
    
    ###########################################################################
    # STEP THE SIMULATION
    sim.step(real_time=True,   # We want our simulation to run in real time (or as close as possible)
              update_vis=True, # We want to update the visualizer each time we step the physics
              update_ani=True  # We want to update the animator each time we step the physics
            )

PRESS ENTER TO START SIMULATION.
PRESS ESC TO QUIT.
PRESS SPACE TO PAUSE/RESUME SIMULATION.
PRESS BACKSPACE TO RESET SIMULATION.
CONTINUING...
RESET
QUITTING...
Termination command detected. Terminating keyboard listener. Goodbye
